In [ ]:
# =============================================================================
# ELEKTRİKLİ ARAÇ SOC TAHMİNİ VE BATARYA YÖNETİMİ PROJESİ
# İlk Üç Adım: Veri Yükleme, EDA ve Model Geliştirme
# =============================================================================

import warnings
warnings.filterwarnings('ignore')

# --- BÖLÜM 1: KÜTÜPHANELERİN YÜKLENMESİ ---
import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

# Grafik ayarları
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_palette("husl")

print("Kütüphaneler yüklendi.")
print(f"TensorFlow: {tf.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

# --- BÖLÜM 2: VERİNİN YÜKLENMESI VE HAZIRLANMASI ---
print("\n" + "="*60)
print("BÖLÜM 2: VERİ YÜKLEME VE ÖN İŞLEME")
print("="*60)

# B0005.mat dosyasını yükle
data_path = "../data/B0005.mat"

# Eğer notebook farklı dizindeyse, mutlak yol kullan
if not os.path.exists(data_path):
    data_path = "C:/Users/aybars/Desktop/case/electric-vehicle-soc/data/B0005.mat"
    
# Eğer hala bulamazsa, mevcut dizinden kontrol et
if not os.path.exists(data_path):
    data_path = "data/B0005.mat"
    
print(f"Veri dosyası yükleniyor: {data_path}")
print(f"Dosya mevcut: {os.path.exists(data_path)}")

try:
    mat_data = scipy.io.loadmat(data_path)
    print(".mat dosyası yüklendi.")
    
    # .mat dosyasının yapısını incele
    print("\nDosya yapısı:")
    for key in mat_data.keys():
        if not key.startswith('__'):
            print(f"  - {key}: {type(mat_data[key])}")
    
    # B0005 verisine erişim
    battery_data = mat_data['B0005']
    print(f"\nBatarya verisi şekli: {battery_data.shape}")
    
    # Cycle verilerine erişim
    cycles = battery_data[0, 0]['cycle'][0]
    print(f"Toplam döngü sayısı: {len(cycles)}")
    
except Exception as e:
    print(f"❌ Veri yükleme hatası: {e}")
    raise

# Veri yapısını daha detaylı inceleyelim
print("\n🔍 Veri yapısı detaylı inceleme:")
print(f"Battery data type: {type(battery_data)}")
print(f"Battery data shape: {battery_data.shape}")

# İlk döngüyü inceleyelim
print(f"\nİlk döngü yapısı:")
first_cycle = cycles[0]
print(f"First cycle type: {type(first_cycle)}")
print(f"First cycle keys: {first_cycle.dtype.names if hasattr(first_cycle, 'dtype') else 'No dtype'}")

# Döngü yapısını anlayalım
print(f"\nDöngü içeriğini inceleyelim:")
try:
    for field_name in first_cycle.dtype.names:
        field_data = first_cycle[field_name]
        print(f"  {field_name}: {type(field_data)} - shape: {field_data.shape if hasattr(field_data, 'shape') else 'No shape'}")
        if field_name == 'type':
            print(f"    Type value: {field_data}")
except Exception as e:
    print(f"Hata: {e}")

# Sadece 'discharge' tipindeki döngüleri filtrele ve veri çerçevesine dönüştür
print("\n🔄 Deşarj döngüleri işleniyor...")

all_cycles_data = []
cycle_count = 0

for i, cycle in enumerate(cycles):
    try:
        # Döngü tipini kontrol et - daha güvenli yaklaşım
        if hasattr(cycle, 'dtype') and 'type' in cycle.dtype.names:
            cycle_type = cycle['type']
            
            # Farklı veri yapılarını kontrol et
            if hasattr(cycle_type, 'flatten'):
                type_str = str(cycle_type.flatten()[0]) if len(cycle_type.flatten()) > 0 else ""
            else:
                type_str = str(cycle_type[0][0]) if len(cycle_type) > 0 else ""
            
            print(f"Döngü {i+1} tipi: '{type_str}'")
            
            if 'discharge' in type_str.lower():
                cycle_count += 1
                
                # Veri dizilerini al - daha güvenli yaklaşım
                data_section = cycle['data']
                
                # Veri yapısına göre uyarla
                if hasattr(data_section, 'flatten') and len(data_section.flatten()) > 0:
                    data_item = data_section.flatten()[0]
                else:
                    data_item = data_section[0][0] if len(data_section) > 0 else None
                
                if data_item is not None:
                    voltage = data_item['Voltage_measured'].flatten()
                    current = data_item['Current_measured'].flatten()
                    temperature = data_item['Temperature_measured'].flatten()
                    time = data_item['Time'].flatten()
                    
                    # Kapasite bilgisini al (eğer varsa)
                    try:
                        capacity_data = data_item['Capacity'].flatten()
                        capacity = capacity_data[0] if len(capacity_data) > 0 else np.nan
                    except:
                        capacity = np.nan
                    
                    # Veri uzunluklarını eşitle (minimum uzunluğa göre)
                    min_length = min(len(voltage), len(current), len(temperature), len(time))
                    
                    if min_length > 0:  # En az bir veri noktası olmalı
                        # Her döngü için DataFrame oluştur
                        cycle_df = pd.DataFrame({
                            'voltage_measured': voltage[:min_length],
                            'current_measured': current[:min_length],
                            'temperature_measured': temperature[:min_length],
                            'time': time[:min_length],
                            'cycle': i + 1,
                            'capacity': capacity
                        })
                        
                        all_cycles_data.append(cycle_df)
                        print(f"  ✅ Döngü {i+1} işlendi - {min_length} veri noktası")
                    
    except Exception as e:
        print(f"⚠️  Döngü {i+1} işlenirken hata: {e}")
        # İlk 5 döngüde hata varsa detayları göster
        if i < 5:
            import traceback
            traceback.print_exc()
        continue

print(f"\n✅ {cycle_count} adet deşarj döngüsü başarıyla işlendi!")

# Tüm döngüleri birleştir
if all_cycles_data:
    final_df = pd.concat(all_cycles_data, ignore_index=True)
    print(f"📊 Birleştirilmiş veri seti boyutu: {final_df.shape}")
else:
    raise ValueError("❌ Hiç deşarj döngüsü bulunamadı!")

# EN ÖNEMLİ ADIM: SOC (State of Charge) hesaplama
print("\n⚡ SOC (State of Charge) hesaplanıyor...")

def calculate_soc(group):
    """Her döngü için SOC hesapla: SOC = 1 - (time / max_time_in_cycle)"""
    max_time = group['time'].max()
    group['SOC'] = 1 - (group['time'] / max_time)
    return group

# Her döngü için SOC hesapla
final_df = final_df.groupby('cycle').apply(calculate_soc).reset_index(drop=True)

print("✅ SOC hesaplaması tamamlandı!")

# Veri setinin genel bilgileri
print("\n" + "="*50)
print("VERİ SETİ GENEL BİLGİLERİ")
print("="*50)

print("\n📋 İlk 5 satır:")
print(final_df.head())

print(f"\n📊 Veri seti bilgileri:")
print(final_df.info())

print(f"\n📈 İstatistiksel özet:")
print(final_df.describe())

print(f"\n🔢 Eksik değer sayıları:")
print(final_df.isnull().sum())

# --- BÖLÜM 3: KEŞİFSEL VERİ ANALİZİ (EDA) ---
print("\n" + "="*60)
print("BÖLÜM 3: KEŞİFSEL VERİ ANALİZİ (EDA)")
print("="*60)

# Grafik 1: Batarya Yaşlanması - Farklı döngülerde voltaj değişimi
print("\n📊 Grafik 1: Batarya Yaşlanması Analizi")

plt.figure(figsize=(14, 8))
cycles_to_plot = [10, 80, 160]
colors = ['blue', 'orange', 'red']

for i, cycle_num in enumerate(cycles_to_plot):
    cycle_data = final_df[final_df['cycle'] == cycle_num]
    if not cycle_data.empty:
        plt.plot(cycle_data['time'], cycle_data['voltage_measured'], 
                color=colors[i], linewidth=2, label=f'Döngü {cycle_num}')

plt.title('Batarya Yaşlanması: Farklı Döngülerde Voltaj Değişimi', fontsize=16, fontweight='bold')
plt.xlabel('Zaman (saniye)', fontsize=12)
plt.ylabel('Voltaj (V)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Analiz: Bu grafik bataryanın yaşlanma sürecini gösteriyor.")
print("İleri döngülerde voltajın daha hızlı düştüğü görülüyor.")

# Grafik 2: Özellik Korelasyonu
print("\n📊 Grafik 2: Özellik Korelasyon Analizi")

# Korelasyon için sayısal sütunları seç
correlation_features = ['voltage_measured', 'current_measured', 'temperature_measured', 'SOC']
correlation_data = final_df[correlation_features].corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_data, dtype=bool))
sns.heatmap(correlation_data, mask=mask, annot=True, cmap='RdYlBu_r', 
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Batarya Özelliklerinin Korelasyon Isı Haritası', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("Analiz: Korelasyon haritası özellikler arası ilişkileri gösteriyor.")
print("SOC ile voltaj arasında güçlü pozitif korelasyon görülmektedir.")

# Grafik 3: Batarya Sağlık Durumu (SOH) - Kapasite Azalması
print("\n📊 Grafik 3: Batarya Sağlık Durumu (SOH) Analizi")

# Döngü başına ortalama kapasite hesapla
capacity_per_cycle = final_df.groupby('cycle')['capacity'].first().reset_index()
capacity_per_cycle = capacity_per_cycle.dropna()

plt.figure(figsize=(14, 8))
plt.plot(capacity_per_cycle['cycle'], capacity_per_cycle['capacity'], 
         'o-', color='darkred', linewidth=2, markersize=4)
plt.title('Batarya Sağlık Durumu: Döngü Sayısına Göre Kapasite Azalması', 
          fontsize=16, fontweight='bold')
plt.xlabel('Döngü Numarası', fontsize=12)
plt.ylabel('Kapasite (Ah)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Analiz: Bu grafik bataryanın sağlık durumunun zamanla değişimini gösteriyor.")
print("Döngü sayısı arttıkça kapasite azalmaktadır.")

# --- BÖLÜM 4: MODEL İÇİN VERİ HAZIRLIĞI ---
print("\n" + "="*60)
print("BÖLÜM 4: MODEL İÇİN VERİ HAZIRLIĞI")
print("="*60)

# Model özellikleri ve hedefi seç
feature_columns = ['voltage_measured', 'current_measured', 'temperature_measured']
target_column = 'SOC'

print(f"📊 Model özellikleri: {feature_columns}")
print(f"🎯 Hedef değişken: {target_column}")

# Eksik değerleri temizle
model_data = final_df[feature_columns + [target_column]].dropna()
print(f"📊 Temizlenmiş veri seti boyutu: {model_data.shape}")

# Veriyi ölçeklendir (0-1 arası)
print("\n⚖️  Veri ölçeklendiriliyor...")
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# Özellikleri ölçeklendir
scaled_features = scaler_features.fit_transform(model_data[feature_columns])
scaled_target = scaler_target.fit_transform(model_data[[target_column]])

print("✅ Veri ölçeklendirme tamamlandı!")

# LSTM için sequence verisi oluştur
print("\n🔄 LSTM için sequence verisi oluşturuluyor...")

SEQ_LENGTH = 10  # 10 zaman adımı

def create_sequences(features, target, seq_length):
    """LSTM için sequence verisi oluştur"""
    X, y = [], []
    for i in range(len(features) - seq_length):
        X.append(features[i:(i + seq_length)])
        y.append(target[i + seq_length])
    return np.array(X), np.array(y)

X_sequences, y_sequences = create_sequences(scaled_features, scaled_target.flatten(), SEQ_LENGTH)

print(f"✅ Sequence verisi oluşturuldu!")
print(f"📊 X şekli (girdi): {X_sequences.shape}")  # [samples, timesteps, features]
print(f"🎯 y şekli (çıktı): {y_sequences.shape}")  # [samples,]

# Eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42, shuffle=False
)

print(f"\n📊 Eğitim seti boyutu: X={X_train.shape}, y={y_train.shape}")
print(f"📊 Test seti boyutu: X={X_test.shape}, y={y_test.shape}")

# --- BÖLÜM 5: LSTM MODELİ GELİŞTİRME, EĞİTİM VE DEĞERLENDİRME ---
print("\n" + "="*60)
print("BÖLÜM 5: LSTM MODELİ GELİŞTİRME VE EĞİTİM")
print("="*60)

# LSTM modelini oluştur
print("🧠 LSTM modeli oluşturuluyor...")

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, len(feature_columns))),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# Modeli derle
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae']
)

print("✅ Model başarıyla oluşturuldu!")
print("\n📋 Model özeti:")
model.summary()

# Modeli eğit
print("\n🎓 Model eğitimi başlıyor...")

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_test, y_test),
    verbose=1,
    shuffle=False
)

print("✅ Model eğitimi tamamlandı!")

# Modeli kaydet - Yeni Keras formatında
model_save_path_h5 = "../src/api/soc_model.h5"
model_save_path_keras = "../src/api/soc_model.keras"

# Eğer dizin yoksa, mutlak yol kullan
if not os.path.exists(os.path.dirname(model_save_path_h5)):
    model_save_path_h5 = "C:/Users/aybars/Desktop/case/electric-vehicle-soc/src/api/soc_model.h5"
    model_save_path_keras = "C:/Users/aybars/Desktop/case/electric-vehicle-soc/src/api/soc_model.keras"

os.makedirs(os.path.dirname(model_save_path_h5), exist_ok=True)

# Her iki formatta da kaydet
model.save(model_save_path_h5)
model.save(model_save_path_keras)
print(f"💾 Model kaydedildi (H5): {model_save_path_h5}")
print(f"💾 Model kaydedildi (Keras): {model_save_path_keras}")

# Model performansını değerlendir
print("\n" + "="*50)
print("MODEL PERFORMANS DEĞERLENDİRMESİ")
print("="*50)

# Test seti üzerinde tahmin yap
y_pred = model.predict(X_test)

# Performans metriklerini hesapla
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# MAPE hesaplama - sadece anlamlı değerler için
mask = y_test > 0.1  # SOC %10'dan büyük değerler için
if np.sum(mask) > 0:
    mape = np.mean(np.abs((y_test[mask] - y_pred.flatten()[mask]) / y_test[mask])) * 100
else:
    mape = mae * 100  # Alternatif metrik

print(f"\n📊 MODEL PERFORMANS METRİKLERİ:")
print(f"   • Mean Absolute Error (MAE): {mae:.6f}")
print(f"   • Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"   • Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Eğitim geçmişini görselleştir
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Loss', color='blue')
plt.plot(history.history['val_loss'], label='Doğrulama Loss', color='red')
plt.title('Model Loss Değişimi', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Eğitim MAE', color='blue')
plt.plot(history.history['val_mae'], label='Doğrulama MAE', color='red')
plt.title('Model MAE Değişimi', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tahmin sonuçlarını görselleştir
plt.figure(figsize=(14, 8))
plot_range = min(500, len(y_test))  # İlk 500 örneği göster
plt.plot(y_test[:plot_range], label='Gerçek SOC', color='blue', alpha=0.7)
plt.plot(y_pred[:plot_range], label='Tahmin Edilen SOC', color='red', alpha=0.7)
plt.title('SOC Tahmin Sonuçları: Gerçek vs Tahmin', fontsize=16, fontweight='bold')
plt.xlabel('Örnek Numarası')
plt.ylabel('SOC (Normalize Edilmiş)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("PROJE TAMAMLANDI")
print("="*60)
print("Tamamlanan Adımlar:")
print("   1. Veri Yükleme ve Ön İşleme")
print("   2. Keşifsel Veri Analizi (EDA)")
print("   3. SOC Tahmin Modeli Geliştirme")
print(f"\n📊 Final Model Performansı:")
print(f"   • MAE: {mae:.6f}")
print(f"   • RMSE: {rmse:.6f}")
print(f"   • MAPE: {mape:.2f}%")
print(f"\nModel kaydedildi: {model_save_path_h5}")
# Analiz tamamlandı
